# Unsupervised Learning with Non-Ignorable Missing Data

Christine Hwang

In [508]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression as Lin_Reg
from sklearn.linear_model import Ridge as Ridge_Reg
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso as Lasso_Reg
from statsmodels.regression.linear_model import OLS
from sklearn.ensemble import RandomForestClassifier
import sklearn.preprocessing as Preprocessing
from sklearn.preprocessing import StandardScaler as Standardize
import itertools as it
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import matplotlib.colors as colors
import scipy as sp
from itertools import combinations
%matplotlib inline
import math

## Summary of Paper

### Overview
Missing data is defined to be non-ignorable if it is not missing at random. Non-ignorable missing data causes the inference based on observed data to lead to biased parameter estimates and can affect the significance of your results. This paper attempts to use probabilistic models to fill in missing data to reduce this bias

### Example
In the scenario where you have netflix movie ratings, you are more likely to rate movies that you really enjoy or really dislike, but rarely rate movies that you are neutral about. Therefore, most of the missing data tends to be from these middle ratings and the distribution of the observed data is shifted to the right from the true distribution of copmletely filled data. In this scenario, the probability of observing a particular response depends on the value itself. Therefore, ignoring missing data can lead to biased parameter estimates. 

### Standard Mixture Model

<img src="GraphicalModel.png">

This standard mixsture model shows a latent variable z behind each observation Y. In this example of a multinomial mixture model, $z_n$ represents the latent variable behind each entry which is pulled from a $\theta_k$ prior. This means that there are k different latent variables that our multinomial distribution can come from. For each observed value given its latent variable, there is a $\beta_{vmz}$ that represents the probability of any observation taking a specific value.

## Interpretation of the Model

Just want to make sure I have the right interpretation of the distribution.

Let us consider the scenario where we represent the netflix movie ratings as a multinomial distribution. Then from the diagram above, $Y_{1n'}$ = the rating star count for movie 1 for person n' and we assume there are n # people. Then if $Y_{im}$ = v, that means that the star count for movie i from person m = v. To represent each $Y_m$ is a multinomial distribution, we will define it as Mult($\theta_j$, M), where $\theta_z$ = P(selecting word v from latent variable z) where M is the total number of ratings.

$z_n$ ~ Mult($\pi$,1) 


$\pi$ = [.3, .2, .1, .4]

This means that the probability of having a latent variable $z_1$ = .3, $z_2$ = .2, etc.


If we represent each $Y_{m}|z_i$ ~ Multi($\theta_z$,N), we view $\theta_z$ to be the probability distribution of ratings for a single movie. For example, $\theta = [.1 .2 .7]$ means that the probability you give the movie 1 star is .1, probability you give it 2 stars is .2, and probability you give it three stars is .7, and you can set your M = N so that it follows a Multinomial($\theta$, N). This is because N people give movie $m$ a rating and therefore the distribution of all the ratings that the N people could have given movie $m$ is Multinoimal given the $\theta$.

Then the $\beta$ estimates $\theta$ is if we view $\theta$ to be the probability distribution of ratings for a single movie. For example, $\theta = [.1 .2 .7]$ means that the probability you give the movie 1 star is .1, probability you give it 2 stars is .2, and probability you give it three stars is .7, and you can set your M = N so that it follows a Multinomial($\theta$, N). Then it would make sense that $Y_{m}$~Multi($\beta$,N) which represents the ratings from N people for the $m^{th}$ movie.

### CPT-v Model

<img src="Model2.png">

We extrapolate from the previous model to show the missing data. $\mu_v$ = $P(R_m = 1 | Y_m = v)$, whic his the probability the value is missing given the true value of the data. In our movie rating example, this probability is higher is $Y_m = 2-3$ because we are less likely to see a rating for a mediocre movie. Need to clarify previous questions before analyzing this model further.

## Medical Application

I hope to apply this model of missing data to medical data. For example, in data analyzing blood pressure or cholesterol, we may see that the probability of seeing missing values for younger, healthier people is higher than for older people because they are less likely to have health complications that require these measurements. Doctors may assume that these measurements are not necessary for the checkup and therefore the data will not be missing at random. In order to use this probabilistic model to fill in missing values, I need to understanding how these categorical variables (if you just convert them to integers) can represent a multinomial distribution. If a person's blood pressure can take the values [10, 50, 100], and our $\theta = [.2 .2 .6]$, does this mean that the person's blood pressure = Multinomial($\theta$, 1)? 

## Baseline Implementation

<img src="EM.png">

$\phi_{zn}$ = posterior distribution of the latent variable z 

$\theta_z$ = probability of that latent variable Z = z

$\beta_{vmz}$ = probability that Y = v given latent variable

$\mu_v$ = probability that $Y_i$ is missing given is has value v.

$\gamma$ and $\lambda$ are intermediate variables without interpretable value.

$\delta = I(y_{mn} = v)$


## Priors

$u_v$ comes from a beta distribution because it is a prior for a Binomial Distribution

$\theta$ comes from a Dirichlet distribution

$\beta$ comes from a dirichlet distribution

### Generate Full Data
I created the probability distribution of movies for there different movies. Movie 1 is a bad movie that is likely to get a lot of 1s. Beta 2 is just an average movie whose ratings will resemble a normal distribution centered at 3. Movie 3 is a great movie that is likely to get very high reviews. Concatentate these ratings to get your full data set of 100 movie ratings for these 3 movies.

## Question:
I don't think I'm coding this correctly so that the reviews are consistent with each person.. Walk through approach and ask if it makes intuitive sense

In [358]:
def simulate_data(z=1, n = 100, m = 3, v = 5):
    
    ### create the different betas
    #########
    # If there are three latent variables, then each movie will have 3 sets of probability distributions
    # because there can be "teen, young adult, old people" categories that have different movie preferences.
    #
    # Input: z = 3, m = 3
    # Output: [ (z1:m1 rating),(z1:m2 rating), (z1:m3 rating), (z2:m1 rating), etc]
    #
    #########
    all_beta = []
    for z_ in range(z):
        beta = []
        for m_ in range(m):
            beta_temp = np.random.dirichlet(np.ones(v),size=1)[0]
            beta.append(beta_temp)
        all_beta.append(beta)
        

    # Generates which latent variable each user comes from    
    z_list = np.random.randint(1,z+1,n)
    complete_data = np.zeros((n,m))
    for i in range(len(z_list)):
        for m_ in range(m):
            complete_data[i][m_] = np.argmax(np.random.multinomial(1,list(all_beta[z_list[i]-1][m_])))+1
    return complete_data, all_beta,z_list

In [359]:
# z=3
# m = 5
# v = 5
# n = 1000
# complete_data,true_beta,z_list = simulate_data(z = z,n=n,m=m)

### Mu Initialization

In the paper in study, we let $\mu_v(s)$ = $s$($v$ − 3) + 0.5, where $s$ is the parameter that controls the strength of the effect. I will set $s$ = .075. 

In terms of the example of a movie rating, It makes more sense that the movie will be missing reviews it is a neutral movie than a bad movie. Therefore, I reversed the probability so that it more resembles a bell shaped distribution where the probability of the rating being missing peaks when star = 3.

In [360]:
# for i in range(v):
#     mu[i] = -.075*abs(i-2)+.5
# mu

### Pattern for Missing Data (R)

Given this, we want to create a matrix that indicates 1 if the data is observed and 0 otherwise.

### Pull $\theta, \beta$ from Dirichlet

Because $\theta$ and $\beta$ are good priors for categorical distribution and multinomial distributions, they are appropriate priors for $\theta$ and $\beta$ because both represent the probability of taking a specific categorical value.

In [361]:
#### proxy for when it found convergence rather than iterating but very volatile results that don't really converge
#### if i set epsilon to be smaller?

def run_EM(theta,beta,phi,gamma,mu,lambda_,v,m,z,n,complete_data,r):
    import time

    start = time.time()
    
    ### store the past
    past_beta = np.zeros((v,m,z))
    past_gamma = np.zeros((m,z,n))
    past_phi = np.zeros((z,n))
    past_mu = np.zeros(v)
    past_lambda_ = np.zeros((v,m,z,n))
    past_theta = np.zeros(z)
    
    counter= 0
    
    eps = .01
#     while(abs(past_beta-beta).sum() > eps and abs(past_gamma-gamma).sum() > eps and abs(past_mu-mu).sum() > eps and
#          abs(past_phi-phi).sum() > eps and abs(past_lambda_-lambda_).sum() > eps and abs(past_theta-theta).sum() > eps): 
    for i in range(100):    

        ### E step
        past_phi = np.copy(phi)
        past_lambda_ = np.copy(lambda_)
        past_theta = np.copy(theta)
        past_beta = np.copy(beta)
        past_gamma = np.copy(gamma)
        past_mu = np.copy(mu)
        ###### lambda
        for v_ in range(v):
            for m_ in range(m):
                for z_ in range(z):
                    for n_ in range(n):
                        lambda_[v_][m_][z_][n_] = ((complete_data[n_][m_]==(v_+1))*mu[v_]*beta[v_][m_][z_])**r[n_][m_]*((1-mu[v_])*beta[v_][m_][z_])**(1-r[n_][m_])


        ###### gamma
        gamma = lambda_.sum(axis = 0)

        ##### vectorized phi
        phi = np.tile(theta, (n,1)).T*gamma.prod(axis = 0)/(np.tile(theta, (n,1)).T*gamma.prod(axis = 0)).sum(axis=0)


        ### M step
        theta = phi.sum(axis=1)/phi.sum()

        #vectorized beta
        for v_ in range(v):
            for m_ in range(m):
                for z_ in range(z):
                    sum_ = 0
                    for n_ in range(n):
                        sum_ += phi[z_][n_]*lambda_[v_][m_][z_][n_]/gamma[m_][z_][n_]
                    beta[v_][m_][z_] = sum_

        beta = beta/phi.sum(axis=1)


        ######mu
        for v_ in range(v):
            num = 0
            denom = 0
            for n_ in range(n):
                for z_ in range(z):
                    for m_ in range(m):
                        num += phi[z_][n_]*r[n_][m_]*lambda_[v_][m_][z_][n_]/gamma[m_][z_][n_]
                        denom += phi[z_][n_]*lambda_[v_][m_][z_][n_]/gamma[m_][z_][n_]
            mu[v_] = num/denom

        if counter % 100 == 0:
            print counter
        counter += 1
    end = time.time()
    print(end - start)  
    return mu,gamma,phi,beta,theta,lambda_

In [362]:
def create_prior(v,m,z,n):
    #mu
    mu = np.ones(v)
    for i in range(v):
        mu[i] = -.075*abs(i-2)+.5
    if z == 1:
        theta = [1]
    else:
        c = ()
        for i in range(z):
            c = (1,)+c
            
        theta = np.random.dirichlet(c,1).reshape((z,))
    c = ()
    for m_ in range(v):
        c = (1,) + c
    beta = np.random.dirichlet(c,(z,m)).transpose()
    
    phi = np.random.random((z,n))
    gamma = np.random.random((m,z,n))
    lambda_ = np.random.random((v,m,z,n))
    
    return mu,theta,beta,phi,gamma,lambda_


In [363]:
def calculate_missing(n,m,complete_data,mu):
    ####### R matrix that decides which values are missing
    r = np.ones((n,m))
    for n_ in range(n):
        for m_ in range(m):
            val = complete_data[n_][m_]
            if np.random.rand(1) < mu[val-1]:
                r[n_][m_] = 0
    return r
                

In [364]:
def missing_value_filled_in(mu,phi,beta,n,m,missing_data,sample,replace = 0):
    
    ### fills in the data
    for n_ in range(n):
        for m_ in range(m):
            latent = np.argmax(phi[:,n_])
            if missing_data[n_][m_] == replace:
                temp = []
                for i in range(sample):
                    temp.append(np.argmax(np.random.multinomial(1, beta[:,m_,latent]))+1)
                missing_data[n_][m_] = np.mean(temp)
       
    return missing_data

In [365]:
z=1
m = 1
v = 7
n = 2417
complete_data,true_beta,z_list = simulate_data(z = z,n=n,m=m)

In [366]:
mu,theta,beta,phi,gamma,lambda_ = create_prior(v,m,z,n)

In [367]:
r = calculate_missing(n,m,complete_data,mu)

/Users/ChrsitineHwang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [369]:
(len(r)-sum(r))/len(r)

array([ 0.42035581])

In [322]:
mu,gamma,phi,beta,theta,lambda_ = run_EM(theta,beta,phi,gamma,lambda_,v,m,z,n,complete_data*r,r)

0
25.3810970783


In [295]:
missing_data = r*complete_data
missing_data

array([[ 3.],
       [ 3.],
       [ 0.],
       ..., 
       [ 4.],
       [ 0.],
       [ 0.]])

In [296]:
model_filled_in = missing_value_filled_in(mu,phi,beta,n,m,np.copy(missing_data),10)
model_filled_in

array([[ 3. ],
       [ 3. ],
       [ 3.3],
       ..., 
       [ 4. ],
       [ 3.9],
       [ 3.5]])

In [232]:
target_var = complete_data[:,0]+complete_data[:,1] + complete_data[:,2]+complete_data[:,3]-complete_data[:,4]+np.random.normal(0,1,len(complete_data[:,0]))

In [233]:
def standard_imputation(missing_data):
    column_names = missing_data.columns
    for col in column_names:
        col_mean = np.mean(missing_data[col])
        for i in range(len(missing_data[col])):
            if missing_data[col][i] == 0:
                missing_data[col][i] = col_mean
    print missing_data.shape
    return missing_data

In [234]:
standard_missing_data = standard_imputation(pd.DataFrame(np.copy(missing_data)))


(10000, 5)


In [235]:
lasso = Lasso_Reg(alpha = 0)
lasso.fit(np.array(model_filled_in),target_var)
print lasso.score(np.array(model_filled_in),target_var)

/Users/ChrsitineHwang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  from ipykernel import kernelapp as app


0.533970536151


In [236]:
lasso = Lasso_Reg(alpha = 0)
lasso.fit(np.array(standard_missing_data),target_var)
print lasso.score(np.array(standard_missing_data),target_var)

/Users/ChrsitineHwang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  from ipykernel import kernelapp as app


0.427496248412


In [237]:
lasso = Lasso_Reg(alpha = 0)
lasso.fit(np.array(complete_data),target_var)
print lasso.score(np.array(complete_data),target_var)

/Users/ChrsitineHwang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  from ipykernel import kernelapp as app


0.901866545763


### Time Markdown
With the previous brute force method for phi, the time it took to run 10,000 iterations was 277.


With the vectorized method for phi, the time it took to run 10,000 iterations was 256.


After vectorizing the denominator of beta, it sped up to 235.

Using the convergence proxy, I clearly don't need 10,000 iterations so maybe vectorizing not entirelly necessary

# Multiple latent variable progress

In [316]:
z=2
m = 5
v = 5
n = 5000
complete_data,true_beta,z_list = simulate_data(z = z,n=n,m=m)
mu,theta,beta,phi,gamma,lambda_ = create_prior(v,m,z,n)
r = calculate_missing(n,m,complete_data,mu)
mu,gamma,phi,beta,theta,lambda_ = run_EM(theta,beta,phi,gamma,mu,lambda_,v,m,z,n,complete_data,r)
missing_data = r*complete_data
model_filled_in = missing_value_filled_in(mu,phi,beta,n,m,np.copy(missing_data),10)
target_var = complete_data[:,0]+complete_data[:,1] + complete_data[:,2]+complete_data[:,3]-complete_data[:,4]+np.random.normal(0,1,len(complete_data[:,0]))


standard_missing_data = standard_imputation(pd.DataFrame(np.copy(missing_data)))
lasso = Lasso_Reg(alpha = 0)
lasso.fit(np.array(model_filled_in),target_var)


/Users/ChrsitineHwang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


0


KeyboardInterrupt: 

In [ ]:
model_filled_in

In [239]:
print "my model,", lasso.score(np.array(model_filled_in),target_var)
lasso = Lasso_Reg(alpha = 0)
lasso.fit(np.array(standard_missing_data),target_var)
print "impute with mean", lasso.score(np.array(standard_missing_data),target_var)
lasso = Lasso_Reg(alpha = 0)
lasso.fit(np.array(complete_data),target_var)
print "complete data model",lasso.score(np.array(complete_data),target_var)

my model, 0.519206335835
impute with mean 0.386073104787


/Users/ChrsitineHwang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  app.launch_new_instance()
/Users/ChrsitineHwang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


complete data model 0.883765549852


# Using prediction for actual real DataSet (mammographic data UCI)

In [519]:
import urllib
raw_data = urllib.urlopen("https://archive.ics.uci.edu/ml/machine-learning-databases/mammographic-masses/mammographic_masses.data")

In [520]:
dataset = np.loadtxt(raw_data,dtype = np.str_, delimiter=",")

In [521]:
dataset = pd.DataFrame(dataset)

In [522]:
dataset = dataset.replace('?','-9')
dataset = dataset.astype(float)

In [523]:
model_2 = fill_missing_data(dataset[2])
model_3 = fill_missing_data(dataset[3])
model_4 = fill_missing_data(dataset[4])
model_5 = fill_missing_data(dataset[5])

0
6.37610197067
0
7.98341393471
0
6.32873106003
0
1.57133793831


In [516]:
def fill_missing_data(missing_data):
    z=1
    m = 1
    v = int(missing_data.max())
    n = len(missing_data)
    
    missing_data = missing_data.reshape((n,m))
    mu_prior,theta_prior,beta_prior,phi_prior,gamma_prior,lambda_prior = create_prior(v,m,z,n)
    r = np.array([ 1-(x== -9) for x in missing_data]).reshape((n,m))
    mu,gamma,phi,beta,theta,lambda_ = run_EM(theta_prior,beta_prior,phi_prior,gamma_prior,mu_prior,lambda_prior,v,m,z,n,missing_data,r)
    model_filled_in = missing_value_filled_in(mu,phi,beta,n,m,np.copy(missing_data),20,replace=-9)
    return model_filled_in

In [461]:
missing_data = dataset[2]

In [463]:
z=1
m = 1
v = 4
n = 961

In [464]:
missing_data = missing_data.reshape((n,m))

In [465]:
mu_prior,theta_prior,beta_prior,phi_prior,gamma_prior,lambda_prior = create_prior(v,m,z,n)
r = np.array([ 1-(x== -9) for x in missing_data]).reshape((n,m))

In [466]:
mu,gamma,phi,beta,theta,lambda_ = run_EM(theta_prior,beta_prior,phi_prior,gamma_prior,mu_prior,lambda_prior,v,m,z,n,missing_data,r)

0
6.40982580185


In [467]:
model_filled_in = missing_value_filled_in(mu,phi,beta,n,m,np.copy(missing_data),20,replace=-9)

In [468]:
missing_data2 = model_filled_in

In [469]:
missing_data = dataset[3]
z=1
m = 1
v = 5
n = 961
missing_data = missing_data.reshape((n,m))
mu_prior,theta_prior,beta_prior,phi_prior,gamma_prior,lambda_prior = create_prior(v,m,z,n)
import math
r = np.array([ 1-(x== -9) for x in missing_data]).reshape((n,m))
mu,gamma,phi,beta,theta,lambda_ = run_EM(theta_prior,beta_prior,phi_prior,gamma_prior,mu_prior,lambda_prior,v,m,z,n,missing_data,r)
model_filled_in = missing_value_filled_in(mu,phi,beta,n,m,np.copy(missing_data),20,replace=-9)
missing_data3 = model_filled_in

0
7.71221899986


In [470]:
missing_data = dataset[4]
z=1
m = 1
v = 4
n = 961
missing_data = missing_data.reshape((n,m))
mu_prior,theta_prior,beta_prior,phi_prior,gamma_prior,lambda_prior = create_prior(v,m,z,n)
import math
r = np.array([ 1-(x== -9) for x in missing_data]).reshape((n,m))
mu,gamma,phi,beta,theta,lambda_ = run_EM(theta_prior,beta_prior,phi_prior,gamma_prior,mu_prior,lambda_prior,v,m,z,n,missing_data,r)
model_filled_in = missing_value_filled_in(mu,phi,beta,n,m,np.copy(missing_data),20,replace=-9)
missing_data4 = model_filled_in

0
6.55804300308


In [ ]:
missing_data = dataset[4]
z=1
m = 1
v = 4
n = 961
missing_data = missing_data.reshape((n,m))
mu_prior,theta_prior,beta_prior,phi_prior,gamma_prior,lambda_prior = create_prior(v,m,z,n)
import math
r = np.array([ 1-(x== -9) for x in missing_data]).reshape((n,m))
mu,gamma,phi,beta,theta,lambda_ = run_EM(theta_prior,beta_prior,phi_prior,gamma_prior,mu_prior,lambda_prior,v,m,z,n,missing_data,r)
model_filled_in = missing_value_filled_in(mu,phi,beta,n,m,np.copy(missing_data),20,replace=-9)
missing_data4 = model_filled_in

In [524]:
df = pd.concat([dataset[0],dataset[1],pd.DataFrame(model_2,columns = ['2']),pd.DataFrame(model_3,columns=['3']),pd.DataFrame(model_4,columns=['4']),dataset[5]],axis=1)

In [525]:
train_indices = np.random.uniform(size=len(dataset[0])) > 1. / 3.

In [526]:
train = dataset[train_indices]
test = dataset[~train_indices]

In [527]:
col2_mean = np.mean(train[2][train[2]>-9])
col3_mean = np.mean(train[3][train[3]>-9])
col4_mean = np.mean(train[4][train[4]>-9])

In [528]:
train[2] = train[2].replace(-9,col2_mean)
train[3] = train[3].replace(-9,col3_mean)
train[4] = train[4].replace(-9,col4_mean)


/Users/ChrsitineHwang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/ChrsitineHwang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/ChrsitineHwang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [529]:
col2_mean = np.mean(test[2][test[2]>-9])
col3_mean = np.mean(test[3][test[3]>-9])
col4_mean = np.mean(test[4][test[4]>-9])
test[2] = test[2].replace(-9,col2_mean)
test[3] = test[3].replace(-9,col3_mean)
test[4] = test[4].replace(-9,col4_mean)


/Users/ChrsitineHwang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ChrsitineHwang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ChrsitineHwang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydat

In [530]:
standard = LogisticRegression()
standard.fit(train[[1,2,3,4,5]],train[0])
standard.score(test[[1,2,3,4,5]],test[0])

0.80327868852459017

In [531]:
train = df[train_indices]
test = df[~train_indices]

In [532]:
model = LogisticRegression()
model.fit(train[[ 1, u'2', u'3', u'4', 5]],train[0])
model.score(test[[1, u'2', u'3', u'4', 5]],test[0])

0.80000000000000004

# Try a dataset with more missing values

In [507]:
import urllib
raw_data = urllib.urlopen("https://archive.ics.uci.edu/ml/machine-learning-databases/horse-colic/horse-colic.data")
dataset = np.loadtxt(raw_data,dtype = np.str_)
dataset = pd.DataFrame(dataset)
dataset = dataset.replace('?','-9')
dataset = dataset.astype(float)
dataset

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,2.0,1.0,530101.0,38.5,66.0,28.0,3.0,3.0,-9.0,2.0,...,45.0,8.4,-9.0,-9.0,2.0,2.0,11300.0,0.0,0.0,2.0
1,1.0,1.0,534817.0,39.2,88.0,20.0,-9.0,-9.0,4.0,1.0,...,50.0,85.0,2.0,2.0,3.0,2.0,2208.0,0.0,0.0,2.0
2,2.0,1.0,530334.0,38.3,40.0,24.0,1.0,1.0,3.0,1.0,...,33.0,6.7,-9.0,-9.0,1.0,2.0,0.0,0.0,0.0,1.0
3,1.0,9.0,5290409.0,39.1,164.0,84.0,4.0,1.0,6.0,2.0,...,48.0,7.2,3.0,5.3,2.0,1.0,2208.0,0.0,0.0,1.0
4,2.0,1.0,530255.0,37.3,104.0,35.0,-9.0,-9.0,6.0,2.0,...,74.0,7.4,-9.0,-9.0,2.0,2.0,4300.0,0.0,0.0,2.0
5,2.0,1.0,528355.0,-9.0,-9.0,-9.0,2.0,1.0,3.0,1.0,...,-9.0,-9.0,-9.0,-9.0,1.0,2.0,0.0,0.0,0.0,2.0
6,1.0,1.0,526802.0,37.9,48.0,16.0,1.0,1.0,1.0,1.0,...,37.0,7.0,-9.0,-9.0,1.0,1.0,3124.0,0.0,0.0,2.0
7,1.0,1.0,529607.0,-9.0,60.0,-9.0,3.0,-9.0,-9.0,1.0,...,44.0,8.3,-9.0,-9.0,2.0,1.0,2208.0,0.0,0.0,2.0
8,2.0,1.0,530051.0,-9.0,80.0,36.0,3.0,4.0,3.0,1.0,...,38.0,6.2,-9.0,-9.0,3.0,1.0,3205.0,0.0,0.0,2.0
9,2.0,9.0,5299629.0,38.3,90.0,-9.0,1.0,-9.0,1.0,1.0,...,40.0,6.2,1.0,2.2,1.0,2.0,0.0,0.0,0.0,1.0


### Analysis

Wow! My code actually works. Our original beta estimates were $[[.5,.2,.1,.1,.1],[.1,.2,.5,.2,.1],[.05,.1,.1,.35,.4]]$. We can see from the $\beta$ estimates that for the first movie, you are more likely to give it a 1 and for the last movie, you are more likely to give it a 5 and for the second movie, you are most likely to give it a 3. That's what our initial beta estimates are!!

# Predictions

The concept behind predicting is that now that we have a converged beta, for each of the missing values, we will pull the rating value $v$ from a multinomial distribution. How do I deal with this when there are latent variables? How do you know which latent variable each perosn is from? I can take the posterior distribution of the phi and take the argmax to identify which latent variable he or she comes from. 

In [102]:
fill_in_data = r*complete_data
for n_ in range(n):
    for m_ in range(m):
        latent = np.argmax(phi[:,n_])
        if fill_in_data[n_][m_] == 0:
            fill_in_data[n_][m_] = np.argmax(np.random.multinomial(1, beta[:,m_,latent]))+1

In [103]:
r[11]

array([ 1.,  1.,  1.])

In [104]:
fill_in_data[11]

array([ 1.,  2.,  2.])

In [105]:
complete_data[11]

array([1, 2, 2])

In [106]:
def fill_in_accuracy(complete,filled,rmse = True):
    if not rmse:
        difference = 0
        total = 0
        for n_ in range(n):
            for m_ in range(m):
                if (complete[n_][m_] + filled[n_][m_] <> 0):
                    total += 1
                    if complete[n_][m_] == filled[n_][m_]:
                        difference += 1
        print "The total number of missing values is", total
        print "The number of accurate filled values is", difference
    else:
        mad = abs(complete_data[np.nonzero((1-r)*complete_data)]-fill_in_data[np.nonzero((1-r)*fill_in_data)]).mean()
        print "the mean absolute deviation is",mad

In [107]:
fill_in_accuracy((1-r)*complete_data,(1-r)*fill_in_data, rmse=False)
fill_in_accuracy((1-r)*complete_data,(1-r)*fill_in_data, rmse=True)

The total number of missing values is 117
The number of accurate filled values is 35
the mean absolute deviation is 1.2905982906


### Future Steps

I had to hard code with for loop for three of the predictors. Working on how to use matrix multiplication but can't see a clear pattern. Also not sure how to use $\beta$ to actually predict the values after the fitting is done. Do I call np.random.multinomial($\beta$, 1) then take the index of the array that has 1 success and fill in that missing value?

## Next Steps
-- after talk with Weiwei on Tuesday, 

    -- create a function handle multiple latent variables. x
    
    -- speed up by vectorizing some of the code x
    
    -- run the loop using epsilon rather than just a lot of loops. 

## Links

Description of Multinomial Mixture Models
http://web.stanford.edu/~lmackey/stats306b/doc/stats306b-spring14-lecture3_scribed.pdf
https://www.cs.princeton.edu/courses/archive/spring12/cos424/pdf/em-mixtures.pdf

My specific Paper
http://www.cs.ubc.ca/~bmarlin/research/presentations/lnimd_group_talk.pdf
https://people.cs.umass.edu/~marlin/research/papers/aistat-lnimd.pdf

Application of my paper
http://ijcai.org/Proceedings/11/Papers/447.pdf
http://www.cs.toronto.edu/~zemel/documents/cfmar-uai2007.pdf
https://pdfs.semanticscholar.org/2845/eda7ce8de14e351d41182f92b73ece8873ef.pdf
https://people.cs.umass.edu/~marlin/research/thesis/cfmlp.pdf